In [327]:
import pandas as pd
import numpy as np

In [234]:
train_x = pd.read_csv("C:/Users/kxj133/Desktop/AV2/train.csv")
test = pd.read_csv("C:/Users/kxj133/Desktop/AV2/test.csv")

train_x['train'] = "Yes"
test['train'] = "No"
test['redemption_status'] = np.nan

train = pd.concat([train_x,test],axis =0,sort = False)

In [235]:
campaign_data = pd.read_csv("C:/Users/kxj133/Desktop/AV2/campaign_data.csv")
coupon_item_mapping = pd.read_csv("C:/Users/kxj133/Desktop/AV2/coupon_item_mapping.csv")
customer_demographics = pd.read_csv("C:/Users/kxj133/Desktop/AV2/customer_demographics.csv")
customer_transaction_data = pd.read_csv("C:/Users/kxj133/Desktop/AV2/customer_transaction_data.csv")
item_data =  pd.read_csv("C:/Users/kxj133/Desktop/AV2/item_data.csv")

### Customer_Demographics

In [236]:
customer_demographics.tail(15)

###Customer above 45 and family_size is 1 (customer_demographics['family_size'] != '1')
customer_demographics = customer_demographics.assign(abv_age_45_fam_size_1 = 0)

customer_demographics['age_range'].value_counts()

customer_demographics.loc[((customer_demographics['family_size'] == '1') & (customer_demographics['age_range']).isin(['70+','56-70','46-55'])),'abv_age_45_fam_size_1'] = 1

### Removing the no_of_children variable because of high missing values
customer_demographics.drop('no_of_children',axis = 1,inplace = True)

customer_demographics['adj_marital_status'] = customer_demographics['marital_status'].where(((customer_demographics['marital_status'] != np.nan) & (customer_demographics['family_size'] != '1')),'Single')
customer_demographics['adj_marital_status'] = customer_demographics['adj_marital_status'].where(((customer_demographics['marital_status'] != np.nan) & (~(customer_demographics['family_size']).isin(['2','3','4','5+']))),'Married')

### Removing the marital_status variable 
customer_demographics.drop('marital_status',axis = 1,inplace = True)

customer_demographics.head(5)

,customer_id,age_range,rented,family_size,income_bracket,abv_age_45_fam_size_1,adj_marital_status
0,1,70+,0,2,4,0,Married
1,6,46-55,0,2,5,0,Married
2,7,26-35,0,3,3,0,Married
3,8,26-35,0,4,6,0,Married
4,10,46-55,0,1,5,1,Single


### Coupon_varaibles

In [237]:
#### Coupon_varaibles

merged_coupon_items = pd.merge(coupon_item_mapping,item_data,how = "left",on = "item_id")

#### BRAND TYPE VARIABLE
brand_type_counts = merged_coupon_items[['coupon_id','brand_type','brand']].groupby(['coupon_id','brand_type']).count().reset_index().rename( columns = {'brand' : "brand_type_counts"})

merged_coupon_items['brand_counts'] = merged_coupon_items['coupon_id'].map(brand_type_counts.groupby(['coupon_id'])['brand_type'].count())

merged_coupon_items['only_brands'] = merged_coupon_items.apply(lambda x : 'local' if (x['brand_type'] == 'Local') and (x['brand_counts'] == 1) else("established" if (x['brand_type'] == 'Established') and (x['brand_counts'] == 1) else "both"),axis =1)

### ITEM CATEGORY
merged_coupon_items['category'].value_counts()

map_object = {'Grocery' : 'Grocery' , 'Pharmaceutical' : 'Pharmacy', 'Natural Products' : 'Natural Products','Meat' : 'Meat','Packaged Meat' : 'Meat', 'Skin & Hair Care' : 'skin care', 'Seafood' : 'seafood','Flowers & Plants' : 'Flowers&plants','Dairy, Juices & Snacks' : 'dairy&juices' }

merged_coupon_items['category_items'] = merged_coupon_items['category'].map(map_object).fillna('others')

category_counts = merged_coupon_items[['coupon_id','category_items','category']].groupby(['coupon_id','category_items']).count().reset_index()

merged_coupon_items['category_counts'] = merged_coupon_items['coupon_id'].map(category_counts.groupby(['coupon_id'])['category'].count())


merged_coupon_items.drop(['item_id','brand','brand_type','category'],axis = 1 , inplace = True)

category_list = merged_coupon_items['category_items'].unique().tolist()
merged_coupon_items['category_items_list'] = merged_coupon_items['coupon_id'].map(merged_coupon_items.groupby('coupon_id')['category_items'].apply(list))

merged_coupon_items.drop_duplicates(subset= ['coupon_id','only_brands'],inplace = True)

for  i in  category_list:
    merged_coupon_items['order_category_' + i] = merged_coupon_items['category_items_list'].apply(lambda x : 1 if i in x else 0)

merged_coupon_items[merged_coupon_items['category_counts'] == 4].head(1)

merged_coupon_items.drop('category_items_list',axis = 1, inplace = True)

In [238]:
merged_coupon_items.head()

,coupon_id,brand_counts,only_brands,category_items,category_counts,order_category_Grocery,order_category_others,order_category_skin care,order_category_Pharmacy,order_category_seafood,order_category_Meat,order_category_dairy&juices,order_category_Natural Products,order_category_Flowers&plants
0,105,1,local,Grocery,1,1,0,0,0,0,0,0,0,0
1,107,1,local,Grocery,1,1,0,0,0,0,0,0,0,0
2,494,1,established,Grocery,1,1,0,0,0,0,0,0,0,0
3,522,1,established,Grocery,1,1,0,0,0,0,0,0,0,0
4,518,1,established,Grocery,1,1,0,0,0,0,0,0,0,0


### Customer Transaction Data Variables

In [239]:
####Customer Transaction data

customer_transaction_data['actual_price'] = customer_transaction_data["selling_price"] + (customer_transaction_data['other_discount'] * (-1)) + (customer_transaction_data['coupon_discount'] * (-1)) 
customer_transaction_data['diff_actual_selling_price'] = customer_transaction_data['actual_price']- customer_transaction_data['selling_price']
customer_transaction_data['ratio_actual_selling_price'] = customer_transaction_data['selling_price']/customer_transaction_data['actual_price'] 
###customer_transaction_data aggregation
#customer_transaction_data.groupby('customer_id')['selling_price'].sum()

max_purchase = customer_transaction_data.groupby(['customer_id'],sort = True)['item_id','selling_price'].max()
max_purchase = max_purchase.rename(columns = {'item_id' : 'max_sell_price_item',"selling_price" : 'max_sell_price'})
max_actual_price_purchase = customer_transaction_data.groupby(['customer_id'],sort = True)['item_id','actual_price'].max()
max_actual_price_purchase = max_actual_price_purchase.rename(columns = {'item_id' : 'max_actual_price_item',"actual_price" : 'max_actual_price'})

max_actual_max_sell = pd.concat([max_actual_price_purchase,max_purchase],axis =1)

max_actual_max_sell = max_actual_max_sell.reset_index()
max_actual_max_sell.drop(['max_actual_price_item'],axis =1,inplace = True)

max_actual_max_sell['max_diff_actual_sell_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['diff_actual_selling_price'].max())
max_actual_max_sell['min_diff_actual_sell_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['diff_actual_selling_price'].min())
max_actual_max_sell['max_ratio_actual_selling_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['ratio_actual_selling_price'].max())
max_actual_max_sell['min_ratio_actual_selling_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['ratio_actual_selling_price'].min())

max_actual_max_sell['diff_max_actual_max_sell_price'] = max_actual_max_sell['max_actual_price'] - max_actual_max_sell['max_sell_price']
max_actual_max_sell['ratio_max_actual_max_sell_price'] = max_actual_max_sell['max_sell_price']/max_actual_max_sell['max_actual_price']

coupon_used_transaction = customer_transaction_data[customer_transaction_data['coupon_discount'] <0]
max_actual_max_sell['purchase_count'] = max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['selling_price'].count())
max_actual_max_sell['coupon_usage_count'] = max_actual_max_sell['customer_id'].map(coupon_used_transaction.groupby('customer_id')['coupon_discount'].count()).fillna(0)
max_actual_max_sell['ratio_purchase_count_discount_count'] = max_actual_max_sell['coupon_usage_count']/max_actual_max_sell['purchase_count']
max_actual_max_sell['coupon_usage_amt'] = max_actual_max_sell['customer_id'].map(coupon_used_transaction.groupby('customer_id')['coupon_discount'].sum()).fillna(0)
max_actual_max_sell['max_coupon_usage_amt'] = max_actual_max_sell['customer_id'].map(coupon_used_transaction.groupby('customer_id')['coupon_discount'].min()).fillna(0)

####in which transaction he used his first coupon
coupon_first_day = customer_transaction_data[["customer_id","date","coupon_discount"]].groupby(['customer_id','date'],sort = True).sum().reset_index().groupby(['customer_id']).head(1).reset_index()
coupon_first_day.drop(['index','date'],axis =1,inplace = True)

max_actual_max_sell = pd.merge(max_actual_max_sell,coupon_first_day,how = "left",on = "customer_id")

### Since the min_diff_actual_sell_price and max_ratio_actual_selling_price is same for all the customers we are removing that that variable
max_actual_max_sell.drop(['min_diff_actual_sell_price','max_ratio_actual_selling_price'],axis =1,inplace = True)

In [241]:
coupon_item_mapping['item_list'] =  coupon_item_mapping['coupon_id'].map(coupon_item_mapping.groupby(["coupon_id"])["item_id"].apply(list))

coupon_item_mapping.drop(['item_id'],axis =1,inplace = True)

coupon_item_mapping.drop_duplicates(subset = ['coupon_id'],inplace= True)

In [242]:
merged_data = pd.merge(train,campaign_data,how = "left",on = "campaign_id" )

merged_data = pd.merge(merged_data,coupon_item_mapping,how = "left",on = "coupon_id")

#merged_data = pd.merge(merged_data,customer_demographics,how = "left" , on = "customer_id")

In [243]:
## transaction data
merged_data['total_purchase_price'] = merged_data['customer_id'].map(customer_transaction_data.groupby('customer_id')['selling_price'].sum())
merged_data = pd.merge(merged_data,max_purchase,how = "left", on = "customer_id")

In [244]:
merged_data['len_item_list'] = merged_data['item_list'].apply(lambda x : len(x))

merged_data.drop('item_list',axis =1,inplace = True)

In [247]:
merged_data = pd.merge(merged_data,max_actual_max_sell,how = "left", on = "customer_id",sort = True)

In [248]:
merged_data['start_date'] = pd.to_datetime(merged_data['start_date'],format = "%d/%m/%y")

merged_data['end_date'] = pd.to_datetime(merged_data['end_date'],format = "%d/%m/%y")

In [249]:
merged_data['campaign_days'] = (merged_data['end_date'] - merged_data['start_date']).dt.days

In [250]:
merged_data['start_date_month_end'] = merged_data['start_date'].dt.is_month_end
merged_data['end_date_month_end'] = merged_data['end_date'].dt.is_month_end
merged_data['start_date_year_end'] = merged_data['start_date'].dt.is_year_end
merged_data['start_date_quarter'] = merged_data['start_date'].dt.quarter

In [252]:
merged_data = pd.merge(merged_data,merged_coupon_items,how = "left",on = "coupon_id")

In [254]:
#### since for few customers customer_demographics details are missing we separating it into two different datas
missing_customer_id = list(set(train['customer_id']).difference(set(customer_demographics['customer_id'])))

final_data_without_customer_demographics = merged_data[merged_data['customer_id'].isin(missing_customer_id)]

final_data_with_customer_demographics = merged_data[~(merged_data['customer_id'].isin(missing_customer_id))]

In [255]:
final_data_with_customer_demographics = pd.merge(final_data_with_customer_demographics,customer_demographics,how = "left",on = 'customer_id')

final_train_data_customer_demographics = final_data_with_customer_demographics[final_data_with_customer_demographics['train'] == 'Yes']
final_test_data_customer_demographics = final_data_with_customer_demographics[final_data_with_customer_demographics['train'] == 'No']

final_train_data_customer_demographics.columns

drop_columns = ['campaign_id','coupon_id','customer_id','start_date','end_date','train']

final_train_data_customer_demographics.drop(drop_columns,axis = 1,inplace = True)
#drop_columns.append('redemption_status')
final_test_data_customer_demographics.drop(drop_columns,axis= 1,inplace = True)
final_test_data_customer_demographics.drop('redemption_status',axis= 1,inplace = True)

final_train_data_customer_demographics.to_csv("C:/Users/kxj133/Desktop/AV2/variables/customer_demographics/final_train_cust.csv",index= False)
final_test_data_customer_demographics.to_csv("C:/Users/kxj133/Desktop/AV2/variables/customer_demographics/final_test_cust.csv",index = False)

In [324]:
final_train_data_without_customer_demographics = final_data_without_customer_demographics[final_data_without_customer_demographics['train'] == 'Yes']
final_test_data_without_customer_demographics = final_data_without_customer_demographics[final_data_without_customer_demographics['train'] == 'No']

In [325]:
final_train_data_without_customer_demographics.to_csv("C:/Users/kxj133/Desktop/AV2/variables/without_customer_demographics/final_train_without_cust.csv")
final_test_data_without_customer_demographics.to_csv("C:/Users/kxj133/Desktop/AV2/variables/without_customer_demographics/final_test_without_cust.csv")